In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
set_id = 76968-1
url = f"https://www.bricklink.com/catalogPG.asp?.S={set_id}"


'https://www.bricklink.com/catalogPG.asp?.S=76967'

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/141.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

if(response.status_code == 200):
    print('success !')
else:
    print('failed :(', response.status_code)
    
response

failed :( 403


<Response [403]>